<h1><center>Laboratorio 8: ¿Superhéroe o Villano? 🦸</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Patricio Ortiz.

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Sebastián Aguirre
- Nombre de alumno 2: Cristóbal Gallardo


### **Link de repositorio de GitHub:** `https://github.com/cristobalgallardo/MDS7202`

## Temas a tratar

- Codificación de texto usando Bag of Words.
- Búsqueda del modelo óptimo de clasificación usando `GridSearch`
- Uso de pipelines.

## Reglas:

- Fecha de entrega: 23/06/2022
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Obtener caracteristicas a partir de texto usando `CountVectorizer`.
- Fijar un pipeline con un modelo base que luego se irá optimizando.
- Comprender como realizar una búsqueda de grilla sobre un conjunto de clasificadores e hiperparámetros usando `GridSearch`.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [ ]:
# Librería Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

# Clasifación
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

# Librería para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
!pip install umap-learn

# Librería para NLP
!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer
nltk.download('stopwords')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.2 MB 65 kB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 5.0 MB/s 
     |████████████████████████████████| 1.1 MB 35.2 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=27bd905808590617bf97432d00c87ba6e745e58e02dd7ebef5624541ac54bed4
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.7-py3-none-any.whl size=54286 sha256=d48f957c35b8d6602025a15341e7ffbaf61095758a193d2c785caac28f7254ba
  Stored in directory: /root/.cache/pip/wheels/7f/2a/f

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# 1. ¿Quien es Bat Cow?

<p align="center">
  <img src="https://i.imgur.com/D9f1RHy.jpg" width="350">
</p>

En vez de estar desarrollando las evaluaciones correspondientes a su curso, su profesor de catedra y su auxiliar discuten acerca la alineación (héroe o villano) del personaje de ficción Bat-Cow. 

El cuerpo docente, no logra ponerse de acuerdo si el personaje es bueno, neutral o malo: el auxiliar plantea que Bat-cow posee una siniestra mirada, intrigante pero común característica de los personajes malvados. 
Por otra parte, extendiendo las ideas de Rousseau, el profesor plantea que tal como los humanos no nacen malos, no existe motivo por el cual una vaca con superpoderes deba serlo.

Sin embargo, ambos concuerdan que es difícil estimar la alineación solo usando los atributos físicos, por lo que creen el análisis debe ser complementado aún más antes de comunicarle los resultados a su estudiantado. Buscando más información, ambos sujetos se percatan de la existencia de un excelente antecedente para estimar la alineación: la historia personal de cada superhéroe o villano.

Es por esto le solicitan que construya y optimice un clasificador basado en texto el cual analice la alineación de cada personaje basado en su historia personal.

Para este laboratorio deben trabajar con los datos `df_comics.csv` y `comics_no_label.csv` subidos a u-cursos. El primero es un conjunto de datos que les servirá para entrenar un modelo de clasificación, mientras que el segundo es un dataset con personajes de ficción no etiquetados a predecir (sí, aquí está la misteriosa Batcow).

Para comenzar cargue los dataset señalados y visualice a través de un head los atributos que poseen cada uno de los dataset.


In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = '/content/drive/MyDrive/Lab_MDS/Laboratorios/Lab8/'
except: 
    print('Ignorando conexión drive-colab')

Mounted at /content/drive


In [ ]:
df_comics = pd.read_csv(path+'df_comics.csv')
df_comics_no_label = pd.read_csv(path+'comics_no_label.csv')
df_comics = df_comics.dropna(subset=['history_text']) # eliminar ejemplos sin historia

In [ ]:
# queda a labor de su equipo hacer el análisis exploratorio
df_comics.head()

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Aayla Secura,Aayla Secura,NaN,8,ayla Secura was a Rutian Twi'lek Jedi Knight (...,NaN,90,40,45,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
df_comics_no_label.head()

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,1,514A (Gotham),Bruce Wayne,NaN,10,He was one of the many prisoners of Indian Hil...,NaN,100,20,30,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,14,Ace Morgan,Kyle Morgan,NaN,7,NaN,"Aviation: Ace is an extremely skilled pilot, ...",85,20,65,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
2,17,A'dal,NaN,NaN,7,"As with most of the naaru, little is known of ...",NaN,85,30,70,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,27,Agent Zero (FOX),David North,NaN,6,"During mid-late 1973, Zero was a member of Tea...",Zero can absorb kinetic energy to further incr...,90,10,25,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,31,Ajax (FOX),Francis,Francis Freeman,7,Ajax (born Francis Freeman) was a human who ga...,Ajax has claimed that the procedure to obtain ...,85,25,45,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0


In [ ]:
df_comics.describe()

,Unnamed: 0,intelligence_score,strength_score,speed_score,durability_score,power_score,combat_score,has_electrokinesis,has_energy_constructs,has_mind_control_resistance,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
count,1285.000000,1285.000000,1285.000000,1285.000000,1285.000000,1285.000000,1285.000000,1228.000000,1228.000000,1228.000000,...,1228.000000,1228.000000,1228.000000,1228.000000,1228.000000,1228.000000,1228.000000,1228.000000,1228.000000,1228.000000
mean,727.744747,78.665370,38.832685,45.898833,55.610895,66.556420,67.805447,0.089577,0.074919,0.074104,...,0.265472,0.307003,0.403909,0.429153,0.437296,0.387622,0.504886,0.540717,0.566775,0.522801
std,415.297480,25.074439,34.220845,29.176614,31.683003,33.329484,29.598315,0.285690,0.263367,0.262047,...,0.441764,0.461439,0.490880,0.495157,0.496255,0.487406,0.500180,0.498542,0.495723,0.499683
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,372.000000,75.000000,10.000000,25.000000,30.000000,40.000000,55.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,728.000000,85.000000,30.000000,40.000000,55.000000,70.000000,75.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
75%,1084.000000,95.000000,65.000000,65.000000,85.000000,100.000000,90.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1449.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
counts = [len(text) for text in df_comics['history_text']]
df = pd.DataFrame(data=counts, columns=['counts'])
fig = px.histogram(df)
fig.show()

In [ ]:
df.describe()

,counts
count,1285.000000
mean,4058.406226
std,8359.824453
min,7.000000
25%,514.000000
50%,1316.000000
75%,4021.000000
max,129594.000000


## 1.1 Obtención de Features y Bag of Words

<p align="center">
  <img src="https://media0.giphy.com/media/eIUpSyzwGp0YhAMTKr/200.gif" width="300">
</p>

Primero que todo, deben obtener un vector de características del atributo `history_text`, utilizando `Bag of Words`. En este atributo se presenta una breve descripción de la historia de cada uno de los personajes de ficción presentes en el dataset. 

Pero... antes de empezar, ¿Que es `Bag of Words`?...

`Bag of Words` es un modelo de conteo utilizado en Procesamiento de Lenguaje Natural (NLP) que tiene como objetivo generar una representación vectorial (vector de características en nuestro cas) para cada documento a través del conteo de las palabras que contienen. 

La siguiente figura muestra un ejemplo de `Bag of Words` en acción:

<p align="center">
  <img src="https://user.oc-static.com/upload/2020/10/23/16034397439042_surfin%20bird%20bow.png" width="500">
</p>

Como pueden ver, el modelo de `Bag of Words` no resulta tan complicado, ¿pero cómo lo aplicamos en python?. 

Como podrán darse cuenta del ejemplo anterior, para facilitar el conteo será necesario transformar cada uno de los documentos en vectores, donde cada una de las posiciones posee un carácter. Este proceso es conocido como **tokenización** y lo podemos realizar de la siguiente forma:

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
docs = ['The teacher rocks like a good rock & roll',
             'the rock is the best actor in the world']


docs_tokenizados = [word_tokenize(doc)  for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world']]

Podemos mejorar un poco más el proceso de tokenización agregando 

- Stemming:  Definimos Stemming como un algoritmo basado en reglas que transforma las palabras a una forma general. Un ejemplo de stemming, es el siguiente:
- Eliminación de Stopwords: Eliminación de palabras muy frecuentes que entorpecen la clasificación (por ejemplo, el, la los, la, etc...)

<p align="center">
  <img src="https://devopedia.org/images/article/218/8583.1569386710.png" width="300">
</p>


In [ ]:
# Definimos algunas stopword que queremos que sean eliminadas
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('spanish')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador = StemmerTokenizer()

# Creamos algunos documentos
docs = ['The teacher rocks like a good rock & roll',
        'the rock is the best actor in the world',
        'New York is a beautiful city']

# Obtenemos el token del primer documento
[tokenizador(doc) for doc in docs]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['the', 'teacher', 'rock', 'like', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['new', 'york', 'is', 'beauti', 'citi']]

In [ ]:
# Comparación con el caso anterior
docs_tokenizados = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['New', 'York', 'is', 'a', 'beautiful', 'city']]

#### Al Estilo Scikit

Scikit implementa `bag of words` a través de la clase `CountVectorizer()` la cual contiene muchas opciones para mejorar la tokenización.

In [ ]:
bow = CountVectorizer(tokenizer= StemmerTokenizer())
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,actor,beauti,best,citi,good,in,is,like,new,rock,roll,teacher,the,world,york
0,1,0,0,0,0,1,0,0,1,0,2,1,1,1,0,0
1,0,1,0,1,0,0,1,1,0,0,1,0,0,3,1,0
2,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,1


Una de las cosas más interesantes que provee son el use de n-gramas, los cuales, en palabras simples, son conjuntos de n-palabras que se concatenan entre si y que se consideran como tokens separados. 

Pensemos en `Nueva York`. Cuando se tokeniza Nueva York, se generan dos tokens independientes que a simple vista no tienen relación: `Nueva` `York`.
Al usar n-gramas (en un rango min=1,max=2) , generamos tanto `Nueva` y `York` como también `Nueva York` como un token independiente.

In [ ]:
bow = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,& roll,actor,actor in,beauti,beauti citi,best,best actor,citi,good,...,teacher,teacher rock,the,the best,the rock,the teacher,the world,world,york,york is
0,1,1,0,0,0,0,0,0,0,1,...,1,1,1,0,0,1,0,0,0,0
1,0,0,1,1,0,0,1,1,0,0,...,0,0,3,1,1,0,1,1,0,0
2,0,0,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1


De los resultados, podemos ver que generamos vectores de conteo para cada una de las palabras que conforman el corpus.  Un punto extra que se agrega en esta obtención de frecuencias son los bigramas, que básicamente son el conjunto de palabras de tamaño de aparecen juntas en el texto.

## Codificando los Super{heroes, villanos}  [0.5 Puntos]

<p align="center">
  <img src="https://c.tenor.com/LkQzw7k5DV4AAAAd/anime-hacking.gif" width="300">
</p>

Conociendo ahora que es el proceso de `bag of words`, aplique este modelo de obtención de caracteristicas de la siguiente forma en un pipeline:

- Utilice el tokenizador entregado.
- Obtenga caracteristicas de los unigramas y bigramas del texto (tal como el ejemplo).

```python
bog = CountVectorizer(tokenizer= StemmerTokenizer(),`
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )
```

Finalmente, aplique `MinMaxScaler()` sobre `atributos_de_interes` y concatene el valor obtenido con el matriz de caracteristicas obtenidas con bag of words.

```python
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
```

No es necesario que obtenga un dataframe en concreto con las características solicitadas. Se le recomienda generar un `ColumnTransformer()` para aplicar las transformaciones solicitadas en un pipeline.

**To-Do:**
- [ ] Obtener a traves de Bag of Words (`CountVectorizer`) caracteristicas del resumen de historia de cada personaje.
- [ ] Aplicar `MinMaxScaler` sobre los atributos de interes.

**Respuesta:**

In [ ]:
#### Código aquí ####
stop_words = stopwords.words('english') # el texto esta en inglés

atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']


ct = ColumnTransformer([
    ('bog', CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2)), 'history_text'),
    ('minmax', MinMaxScaler(), atributos_de_interes)
])

pipe_cumic = Pipeline([('preprocessing', ct)])
                           
pipe_cumic.fit_transform(df_comics)

<1285x333931 sparse matrix of type '<class 'numpy.float64'>'
	with 794027 stored elements in Compressed Sparse Row format>

## 1.2 Diseño de Baseline y  Primer Entrenamiento  [1 Puntos]


<p align="center">
  <img src="https://pa1.narvii.com/6374/9eaec1b7bf9157334151452a669516f9a78b954c_hq.gif" width="300">
</p>



Genere un Pipeline con las caracteristicas solicitadas en la sección 1.1, un selector de mejores features `SelectPercentile` con métrica `f_classif` y percentile=90 y un clasificador `MultinomialNB()` por defecto.

Luego, separe el conjunto de datos en un conjunto de entrenamiento y prueba, donde las etiquetas estará dado por el atributo `alignment`. 

Entrene el modelo y reporte el desempeño con un `classification_report`. ¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.

Finalmente, compare el modelo entrenado con un modelo Dummy estratificado y responda: ¿El clasificador entrenado es mejor que el dummy que entrega respuestas al azar?

**To-do:**
- [ ] Realizar un pipeline con las caracteristicas solicitadas en 1.1, ejecutar holdout y aplicar un clasificador `MultinomialNB()`.
- [ ] Entrenar el pipeline, calcular el `classification_report` asociado y comentar los resultados.
- [ ] Entrenar un `DummyClassifier` con estrategia `statified`, calcular el `classification_report` asociado y comentar que implican los scores obtenidos en comparación con los resultados del baseline.

**Respuesta:**

In [ ]:
#### Código aquí ####


# bow y minmax
ct = ColumnTransformer([
    ('bog', CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2)), 'history_text'),
    ('minmax', MinMaxScaler(), atributos_de_interes)
])

# preprocesar->seleccionar->clasificar
pipe = Pipeline([('preprocessing', ct),
                       ('selector', SelectPercentile(score_func = f_classif, percentile = 90)),
                       ('clf',  MultinomialNB())])
                           


ft = df_comics.drop('alignment',axis=1) # características
label = df_comics['alignment'] # labels

# train test split
X_train, X_test, y_train, y_test = train_test_split(ft, label, test_size=0.3, random_state=0, shuffle=True, stratify=label)

# fit
pipe.fit(X_train,y_train)

# predict
y_pred= pipe.predict(X_test)

# report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         Bad       0.67      0.12      0.21       129
        Good       0.60      0.97      0.74       223
     Neutral       1.00      0.03      0.06        34

    accuracy                           0.61       386
   macro avg       0.76      0.38      0.34       386
weighted avg       0.66      0.61      0.50       386



In [ ]:
from sklearn.dummy import DummyClassifier  
Dummy_pipe = Pipeline([('clf',  DummyClassifier(strategy = 'stratified', random_state  = 0))])

Dummy_pipe.fit(X_train,y_train)

# predict
y_pred = Dummy_pipe.predict(X_test)

# report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         Bad       0.35      0.34      0.35       129
        Good       0.59      0.59      0.59       223
     Neutral       0.05      0.06      0.06        34

    accuracy                           0.46       386
   macro avg       0.33      0.33      0.33       386
weighted avg       0.46      0.46      0.46       386




El clasificador entrenado es mejor en terminos de accuracy, pero sus métricas promedio macro son casi identicas, fijandose en el f1 de cada clase podemos ver que el clasificador solo mejora en la clase con más soporte pero empeora o se mantiene en las otras, el bajo f1 indica que el clasificador fue sesgado por el desbalance de clases pero tiene resultados parecidos al Dummy.

¿Nos recomendaría predecir la alineación de BatCow con este clasificador?: No, porque sus métricas son tan buenas como adivinar con el Dummy.

## 1.3 Busqueda del Mejor Modelo con Grid Search [4 Puntos]

<p align="center">
  <img src="https://media1.tenor.com/images/70fdfeea52a8e2e4505498c230a0d2f9/tenor.gif?itemid=5134219" width="250">
</p>

No conformes con el rendimiento obtenido en la sección 1.2, el cuerpo docente les pide que realicen un **`HalvingGridSearchCV`** con diferentes parámetros para mejorar el rendimiento de la clasificación. Para esto, se le solicita que defina:

- Tres clasificadores distintos en donde varie sus parámetros. Considere usar modelos clásicos como también los basados en ensamblaje.
- Modificar `n-gram` range del `CountVectorizer` probando `(1,1), (1,2) y (1,3)`. Examinar también los otros parámetros de CountVectorizer como por ejemplo `max_df`, `min_df`, etc... ([Documentación aquí](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html))
- Seleccionar las columnas que contribuyen con la mayor información para la clasificación con `SelectPercentile` en los percentiles `[20, 40, 60, 80]` (puede usar la métrica que usted quiera).
- Reporte la mejor combinación encontrada y justifique por qué cree que es la mejor según el clasificador usado, la cantidad de columnas seleccionadas y los parámetros de CountVectorizer seleccionados por GridSearch.

A continuación, un ejemplo de parametros para GridSearch para una búsqueda de 3 clasificadores distintos:

```python
params = [
       # clasificador 1 + hiperparámetros
       {'clf': classificator1(),
        'clf__penalty': ['ovr'],
       # clasificador 1 + hiperparámetros    
       {'clf': classificator2(),
        'clf__n_estimators': [200]},
       # clasificador 1 + hiperparámetros
       {'clf': classificator3(),
        ...
       }
       ]
```

**Nota 1**: Puede ver los parámetros modificables aplicando el método get_params() sobre su pipeline. Ver la clase de GridSearch para mayor información sobre la sintáxis de las grillas.

**Nota 2**: Recuerde inicializar los clasificadores con un random state definido.

**Nota 3**: Puede usar en `HalvingGridSearchCV` el parámetro `verbose=10` para ver que GridSearch le indique el estado de su ejecución.

**Nota 3:** El GridSearch puede tomar tiempos de búsqueda exorbitantes, por lo que se le recomienda no agrandar mucho el espacio de búsqueda, dejar corriendo el código y tomarse un tecito.

**Respuesta:**

In [ ]:
#### Código aquí ####
# parámetros de la pipeline
pipe.get_params()

{'clf': MultinomialNB(),
 'clf__alpha': 1.0,
 'clf__class_prior': None,
 'clf__fit_prior': True,
 'memory': None,
 'preprocessing': ColumnTransformer(transformers=[('bog',
                                  CountVectorizer(max_df=0.8, min_df=3,
                                                  ngram_range=(1, 2),
                                                  strip_accents='ascii',
                                                  tokenizer=<__main__.StemmerTokenizer object at 0x7f989b42dc90>),
                                  'history_text'),
                                 ('minmax', MinMaxScaler(),
                                  ['intelligence_score', 'strength_score',
                                   'speed_score', 'durability_score',
                                   'power_score', 'combat_score'])]),
 'preprocessing__bog': CountVectorizer(max_df=0.8, min_df=3, ngram_range=(1, 2), strip_accents='ascii',
                 tokenizer=<__main__.StemmerTokenizer object at 0x7f

In [ ]:
"""
max_df: float in range [0.0, 1.0] or int, default=1.0
When building the vocabulary ignore terms that have a document frequency
 strictly higher than the given threshold (corpus-specific stop words).
  If float, the parameter represents a proportion of documents, integer absolute
   counts. This parameter is ignored if vocabulary is not None.

min_df: float in range [0.0, 1.0] or int, default=1
When building the vocabulary ignore terms that have a document frequency
 strictly lower than the given threshold. This value is also called cut-off
  in the literature. If float, the parameter represents a proportion of
   documents, integer absolute counts. This parameter is ignored if vocabulary
    is not None.
"""

params  = [
    {'clf': [DecisionTreeClassifier(random_state = 0)],
     'clf__criterion' : ['gini', 'entropy'],
     'clf__class_weight': [None,'balanced'],
     'preprocessing__bog__strip_accents': ['ascii'], # eliminar acentos
     'preprocessing__bog__ngram_range':[(1,1),(1,2),(1,3)],
     'preprocessing__bog__min_df':[5], # eliminar los que aparecen en menos de 5 documentos
     'preprocessing__bog__max_df':[0.8, 1.0], # eliminar los que aparecen en 80% o 100% de los datos
     'selector__percentile': [20, 40, 60, 80]},

    {'clf': [LogisticRegression(random_state = 0, max_iter=100, solver = 'liblinear')],
     'clf__C' : [0.1, 1.0],
     'clf__class_weight': [None,'balanced'],
     'preprocessing__bog__strip_accents': ['ascii'], # eliminar acentos
     'preprocessing__bog__ngram_range': [(1,1),(1,2),(1,3)],
     'preprocessing__bog__min_df':[5], # eliminar los que aparecen en menos de 5 documentos
     'preprocessing__bog__max_df':[0.8, 1.0], # eliminar los que aparecen en 80% o 100% de los datos
     'selector__percentile': [20, 40, 60, 80]},

    {'clf': [RandomForestClassifier(random_state = 0)],
     'clf__n_estimators': [50, 100],
     'clf__class_weight': [None,'balanced'],
     'preprocessing__bog__strip_accents': ['ascii'], # eliminar acentos
     'preprocessing__bog__ngram_range':[(1,1),(1,2),(1,3)],
     'preprocessing__bog__min_df':[5], # eliminar los que aparecen en menos de 5 documentos
     'preprocessing__bog__max_df':[0.8, 1.0], # eliminar los que aparecen en 80% o 100% de los datos
     'selector__percentile': [20, 40, 60, 80]}
     ]

In [ ]:
from sklearn.experimental import enable_halving_search_cv 
from sklearn.model_selection import HalvingGridSearchCV
hgs  = HalvingGridSearchCV(pipe, params, scoring = 'f1_macro', verbose=10, n_jobs = -1)
# Toma 1:20
hgs.fit(X_train,y_train)

n_iterations: 4
n_required_iterations: 6
n_possible_iterations: 4
min_resources_: 30
max_resources_: 899
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 288
n_resources: 30
Fitting 5 folds for each of 288 candidates, totalling 1440 fits
----------
iter: 1
n_candidates: 96
n_resources: 90
Fitting 5 folds for each of 96 candidates, totalling 480 fits
----------
iter: 2
n_candidates: 32
n_resources: 270
Fitting 5 folds for each of 32 candidates, totalling 160 fits
----------
iter: 3
n_candidates: 11
n_resources: 810
Fitting 5 folds for each of 11 candidates, totalling 55 fits


HalvingGridSearchCV(estimator=Pipeline(steps=[('preprocessing',
                                               ColumnTransformer(transformers=[('bog',
                                                                                CountVectorizer(max_df=0.8,
                                                                                                min_df=3,
                                                                                                ngram_range=(1,
                                                                                                             2),
                                                                                                strip_accents='ascii',
                                                                                                tokenizer=<__main__.StemmerTokenizer object at 0x7f989b42dc90>),
                                                                                'history_text'),
                                 

In [ ]:
print(hgs.best_score_)
print(hgs.best_params_)

0.47945410232844193
{'clf': LogisticRegression(class_weight='balanced', random_state=0, solver='liblinear'), 'clf__C': 1.0, 'clf__class_weight': 'balanced', 'preprocessing__bog__max_df': 1.0, 'preprocessing__bog__min_df': 5, 'preprocessing__bog__ngram_range': (1, 1), 'preprocessing__bog__strip_accents': 'ascii', 'selector__percentile': 60}


In [ ]:
y_pred = hgs.predict(X_test) 
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         Bad       0.53      0.55      0.54       129
        Good       0.73      0.77      0.75       223
     Neutral       0.33      0.15      0.20        34

    accuracy                           0.64       386
   macro avg       0.53      0.49      0.50       386
weighted avg       0.63      0.64      0.63       386



Al realizar cross validation el clasificador mejora sus métricas considerablemente en espacial para las clases Neutral  y Bad (mejora considerablemente el recall respecto al baseline), aún así la clase Neutral obtiene malos resultados, posiblemente por las pocas isntancias de esta clase durante entrenamiento.

```
Justificación aquí
```

Los mejores parámetros del clasificador fueron:

```
{'clf': LogisticRegression(class_weight='balanced', random_state=0,
solver='liblinear'), 'clf__C': 1.0, 'clf__class_weight': 'balanced', 
'preprocessing__bog__max_df': 1.0, 'preprocessing__bog__min_df': 5, 
'preprocessing__bog__ngram_range': (1, 1),
 'preprocessing__bog__strip_accents': 'ascii',
  'selector__percentile': 60}
```

La regresion logística con pesos balanceados resulta tener los mejores resultados en terminos de macro f1, era de esperarse que el caso balanceado mejorara esta métrica debido al desbalance de clases. Además el clasificador seleccionado tiene una regularización L2 moderada (C = 1.0) por lo que se utilizan más las características respecto al otro caso (C = 0.1). 

Una razón que podría explicar por que gana la regresión sobre los árboles es que la clasificación es que basta que las cracterísticas sean ocupadas linealmente o bien que el RF no tiene una cantidad apropiada de estiamadores o profundidad. 

El BOW utiliza un n-gramas de tamaño 1 y filtra las palabras que aparecen en 80% de los datos o más. Finalmente el seleccionador de características toma 60% de las características con mayor ANOVA F, esto en conjunto al valor de max_df indica que no se necesitan todas las palabras del bow para realizar una predicción y que además no favorece la clasificación añadir bi-gramas ni tri-gramas.


## 1.4 Predicción del datos sin etiquetado  [0.5 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/DolotxUUYAAbg7f.jpg" width="350">
</p>


LLego el momento de predecir 
`Vergil`, `Gorilla Girl` y `Batcow`


**Nota:** Recuerde que pueden existir campos vacios en `history_text`, por lo que se les recomienda borrar los nan.

**Respuesta:**

In [ ]:
# Mejores parámetros encontrados

best_params =  {'clf': LogisticRegression(class_weight='balanced', random_state=0, solver='liblinear'),
 'clf__C': 1.0, 'clf__class_weight': 'balanced', 'preprocessing__bog__max_df': 1.0,
  'preprocessing__bog__min_df': 5, 'preprocessing__bog__ngram_range': (1, 1),
   'preprocessing__bog__strip_accents': 'ascii', 'selector__percentile': 60}



new_pipe = Pipeline([('preprocessing', ct),
                       ('selector', SelectPercentile()),
                       ('clf',  None)])

new_pipe = new_pipe.set_params(**best_params)

new_pipe.fit(X_train, y_train)
y_pred = new_pipe.predict(X_test) 
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         Bad       0.53      0.55      0.54       129
        Good       0.73      0.77      0.75       223
     Neutral       0.33      0.15      0.20        34

    accuracy                           0.64       386
   macro avg       0.53      0.49      0.50       386
weighted avg       0.63      0.64      0.63       386



In [ ]:
#### Código aquí ####
df_comics_no_label = df_comics_no_label.dropna(subset=['history_text']) # dropear nan 



df_comics_no_label['alignment'] = new_pipe.predict(df_comics_no_label)

In [ ]:
df_comics_no_label.loc[df_comics_no_label.name.isin(['Vergil', 'Gorilla Girl','Batcow']), ['name', 'alignment']]

,name,alignment
16,Batcow,Bad
40,Gorilla Girl,Bad
78,Vergil,Good
83,Batcow,Bad


<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>